In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets

In [2]:
train_ratio = 0.8
rand_std = 0.030
learning_rate = 0.001
epoch_count = 100
report_period = 10
random_fix = True

In [3]:
iris_dataset = datasets.load_iris()

data = iris_dataset.data
target = iris_dataset.target
target_names = iris_dataset.target_names

#print("dimension: data{}, target{}, target_names{}".
#    format(data.shape, target.shape, target_names.shape))
#print(target_names)
#print(data[:5])
#print(target[-5:])

In [4]:
if random_fix: np.random.seed(1234)

data_count = len(data)
train_count = int(data_count * train_ratio)
test_count = data_count - train_count
    
indices = np.arange(data_count)
np.random.shuffle(indices)
    
train_data = data[indices[0:train_count]]
train_target = target[indices[0:train_count]]
    
test_data = data[indices[train_count:data_count]]
test_target = target[indices[train_count:data_count]]

In [5]:
input_dim, output_dim = 3, 1

def get_test_data():
    test_X = test_data[:, 0:3]
    test_Y = test_data[:, 3:4]

    return test_X, test_Y

def get_train_data(batch_size, nth):
    global indices
    
    if nth == 0:
        indices = np.arange(train_count)
        np.random.shuffle(indices)
        
    from_idx = nth * batch_size
    to_idx = (nth + 1) * batch_size
    
    train_X = train_data[indices[from_idx:to_idx], 0:3]
    train_Y = train_data[indices[from_idx:to_idx], 3:4]   
    
    return train_X, train_Y

In [6]:
def init_rand_normal(in_dim, out_dim):
    if not random_fix:
        init = tf.random_normal([in_dim, out_dim], stddev=rand_std)
    else:
        init_64 = np.random.normal(0, rand_std, [in_dim, out_dim])
        init = init_64.astype('float32')

    return init

In [7]:
x = tf.placeholder("float", [None, input_dim])
y = tf.placeholder("float", [None, output_dim])

if random_fix: np.random.seed(9876)

w = tf.Variable(init_rand_normal(input_dim, output_dim))
b = tf.Variable(tf.zeros([output_dim]))

In [8]:
output = tf.matmul(x, w) + b

In [9]:
loss = tf.reduce_mean(tf.pow(output-y, 2))
accuracy = 1 - tf.reduce_mean(tf.abs(output-y) / y)

In [10]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

In [11]:
def train_and_test(batch_size=0):
    if batch_size == 0: batch_size = train_count
    batch_count = int(train_count / batch_size)
    
    test_X, test_Y = get_test_data()
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    if random_fix: np.random.seed(1945)
    
    for epoch in range(epoch_count):
        costs = []
        accs = []
        for n in range(batch_count):
            train_X, train_Y = get_train_data(batch_size, n)
            _, cost, acc = sess.run([train_op, loss, accuracy], \
                                    feed_dict={x:train_X, y:train_Y})
            costs.append(cost)
            accs.append(acc)
            
        if (epoch+1) % report_period == 0:
            acc = sess.run(accuracy, feed_dict={x:test_X, y:test_Y})
            print("Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f}". \
                  format(epoch+1, np.mean(costs), np.mean(accs), acc))
            
    final_acc = sess.run(accuracy, feed_dict={x:test_X, y:test_Y})
    print("\nFinal Test: final accuracy = {:5.3f}".format(final_acc))
    
    sess.close()

In [12]:
train_and_test(10)

Epoch 10: cost=0.147, accuracy=0.155/-0.161
Epoch 20: cost=0.089, accuracy=0.399/0.168
Epoch 30: cost=0.065, accuracy=0.574/0.348
Epoch 40: cost=0.056, accuracy=0.660/0.469
Epoch 50: cost=0.052, accuracy=0.721/0.518
Epoch 60: cost=0.050, accuracy=0.753/0.571
Epoch 70: cost=0.049, accuracy=0.757/0.599
Epoch 80: cost=0.048, accuracy=0.773/0.596
Epoch 90: cost=0.048, accuracy=0.766/0.618
Epoch 100: cost=0.048, accuracy=0.780/0.624

Final Test: final accuracy = 0.624


In [13]:
train_and_test()

Epoch 10: cost=0.428, accuracy=0.131/-0.182
Epoch 20: cost=0.258, accuracy=-0.049/-0.428
Epoch 30: cost=0.232, accuracy=-0.085/-0.482
Epoch 40: cost=0.218, accuracy=-0.071/-0.467
Epoch 50: cost=0.205, accuracy=-0.041/-0.432
Epoch 60: cost=0.194, accuracy=-0.008/-0.392
Epoch 70: cost=0.184, accuracy=0.024/-0.353
Epoch 80: cost=0.174, accuracy=0.056/-0.315
Epoch 90: cost=0.165, accuracy=0.087/-0.278
Epoch 100: cost=0.157, accuracy=0.116/-0.243

Final Test: final accuracy = -0.243


In [14]:
train_and_test(1)

Epoch 10: cost=0.052, accuracy=0.774/0.622
Epoch 20: cost=0.048, accuracy=0.782/0.714
Epoch 30: cost=0.049, accuracy=0.774/0.713
Epoch 40: cost=0.047, accuracy=0.771/0.722
Epoch 50: cost=0.045, accuracy=0.772/0.670
Epoch 60: cost=0.043, accuracy=0.771/0.676
Epoch 70: cost=0.043, accuracy=0.763/0.646
Epoch 80: cost=0.043, accuracy=0.776/0.617
Epoch 90: cost=0.043, accuracy=0.773/0.703
Epoch 100: cost=0.042, accuracy=0.766/0.710

Final Test: final accuracy = 0.710
